# Building blocks of Clojure

## Metadata

In [5]:
; Using the with-meta function to add metadata to a map
(def untrusted (with-meta 
    {:command "delete-table" :subject "users"}
    {:safe false :io true})) ; Metadata

untrusted

{:command "delete-table", :subject "users"}

In [8]:
; Using the '^' macro to add metadata to a map

(def untrusted 
    ^{:safe false :io true} 
    {:command "delete-table" :subject "users"})

untrusted

{:command "delete-table", :subject "users"}

In [7]:
; If using the '^' macro with a list, 
; the metadata won't take effect at runtime

(def untrusted-not-working 
    ^{:safe false :io true} 
    (hash-map :command "delete-table" :subject "users"))


#'user/untrusted-not-working

In [9]:
; Metadata doesn't affect value equality

(def trusted {:command "delete-table" :subject "users"}) 
(= trusted untrusted) 

true

In [11]:
; Using the meta function to examine metadata
(meta untrusted)


{:safe false, :io true}

In [13]:
; When new values are created from those that have metadata, 
; the metadata is copied over to the new data

(def still-untrusted (assoc untrusted :complete? false))
(meta still-untrusted)


{:safe false, :io true}

In [15]:
; Functions and macros can also be defined with metadata

(defn ^{:safe true 
        :console true
        :doc "testing metadata for functions"} 
    testing-meta
    [] 
    (println "Hello from meta!"))


#'user/testing-meta

In [16]:
; Trying to use the meta function directly to check that 
; the metadata was set correctly won't work

(meta testing-meta)

nil

In [17]:
; To access the metadata,  pass the testing-meta var to the meta function.
(meta (var testing-meta))

{:ns #namespace[user], :name testing-meta, :file "NO_SOURCE_PATH", :column 1, :line 3, :console true, :safe true, :arglists ([]), :doc "testing metadata for functions"}

Defining a function without Java type hints

In [19]:
; Setting a warning to alert when "Reflection" is used
(set! *warn-on-reflection* true) 

(defn string-length [x] (.length x))


Reflection warning, NO_SOURCE_PATH:4:25 - reference to field length can't be resolved.


#'user/string-length

In [28]:
; Function call without type hints (slow)

(time (reduce + (map string-length (repeat 10000 "12345"))))

"Elapsed time: 159.8873 msecs"


50000

Defining a function with Java type hints

In [25]:
(defn fast-string-length [^String x] (.length x)) 


#'user/fast-string-length

In [33]:
; Function call with type hints (fast)

(time (reduce + (map fast-string-length (repeat 10000 "12345"))))

"Elapsed time: 3.4192 msecs"


50000

In [32]:
; The type hints of args is stored as metadata

(meta (first (first (:arglists (meta #'fast-string-length))))) 


{:tag String}

Type hinting primitive types

In [38]:
; Getting the Java Class name of BigDecimal

(defn array-type [klass]
    (.getName (class (make-array klass 0)))) 

(array-type BigDecimal)

"[Ljava.math.BigDecimal;"

In [40]:
; Using the name "[Ljava.math.BigDecimal;" as type hint

(def bigdec-arr 
    ^"[Ljava.math.BigDecimal;" 
    (into-array BigDecimal [1.0M]))


#'user/bigdec-arr

## Java exceptions: try and throw


In [43]:
; A function that calculates the average of a collection of numbers

(defn average [numbers]
    (let [total (apply + numbers)]
        (/ total (count numbers))))

#'user/average

In [44]:
; Calling the average function with an empty sequence throws an exception

(average [])

Execution error (ArithmeticException) at user/average (REPL:5).
Divide by zero


class java.lang.ArithmeticException: 

In [46]:
; Wrapping the last function with a try/catch expression

(defn safe-average [numbers]
    (let [total (apply + numbers)]
        (try
            (/ total (count numbers))
        (catch ArithmeticException e 
            (println "Divided by zero!")
        0))))

(safe-average [])

Divided by zero!


0

In [47]:
; The general form of using try/catch/finally is straightforward:

"""
(try expr* catch-clause* finally-clause?)
"""

""

In [48]:
; Using several catch and a finally clauses

(try
    (print "Attempting division... ")
    (/ 1 0)
(catch RuntimeException e "Runtime exception!") 
(catch ArithmeticException e "DIVIDE BY ZERO!")  
(catch Throwable e "Unknown exception encountered!") 
(finally 
    (println "done.")))


Attempting division... done.


"Runtime exception!"

In [51]:
; The finally clause can be used without catch clauses
(try
    (print "Attempting division... ")
    (/ 1 0)
(finally
    (println "done.")))


Attempting division... done.


Execution error (ArithmeticException) at user/eval4183 (REPL:4).
Divide by zero


class java.lang.ArithmeticException: 

In [52]:
; Throwing exceptions
(throw (Exception. "this is an error!"))

Execution error at user/eval4185 (REPL:2).
this is an error!


class java.lang.Exception: 